# Pyomo Setup and Library Installation

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
#!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

     |████████████████████████████████| 11.1 MB 5.1 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 


In [ ]:
import pandas as pd

Uploading the excel file containing our input data which will be further used for optimization.
=

In [ ]:
# Importing the input data into Google Colab
from google.colab import files
uploaded = files.upload()

Saving Stock Portfolio Optimization v1.xlsx to Stock Portfolio Optimization v1.xlsx


In [ ]:
import io
df = pd.read_excel(uploaded['Stock Portfolio Optimization v1.xlsx'],sheet_name='Stocks(For Python)')

####Exploring the data
The input data contains information about 27 different stocks belonging to 9 different sectors. The data includes each stock's current price and it's future price(i.e. price after 3 months). 

In [ ]:
df.head()

,Company,Sector,Current_Price,Future_Price,Returns
0,TCS,IT services,41.090854,41.835366,0.018119
1,Infosys,IT services,19.777439,20.121951,0.017419
2,Wipro,IT services,4.962195,5.040854,0.015852
3,Bajaj Finance,Finance,80.551220,82.314024,0.021884
4,L&T Finance,Finance,1.126829,1.140244,0.011905


In [ ]:
from pandas.tseries.frequencies import unique
all_sector = unique(df['Sector'])
all_sector.tolist()

['IT services',
 'Finance',
 'Power Generation',
 'Private Banks',
 'Electric equipment',
 'Healthcare',
 'Iron and Steel',
 'Life and Health insurance',
 'Telecommunication services']

Then defining all the parameters required to create and optimize the model based on different constraints.

In [ ]:
from numpy.lib.function_base import average

num_stocks = len(df)

sector_constraint = [2,2,3,2,3,2,2,3,2]
budget = 50000
return_min = average(df['Returns']).tolist()
indv_budget=[]
for i in range(num_stocks):
  a=budget/10
  indv_budget.append(a)

indv_unit=[]
for i in range(num_stocks):
  b=0.01
  indv_unit.append(b)

Defined a function that would return the number of certain stocks to but and how much to invest in those stocks. Also returns our total returns and total profits.

Our objective is to maximise the profit of our portfolio.
<br> Also added the following constraints-<br?

1) Total amount invested should be within the pre defined budget.
<br>2) Average return of portfolio after optimization should be greater than the sum of average individual returns.
<br>3) Amount invested in each stock should be less than 10% of total budget.

Since this is a non linear problem we have implemented ipopt as our solver.

In [ ]:
def solveproblem(company,all_sector,current_price,future_price):
  #defining the optimization model
  model = ConcreteModel()
  
  #defining the decision variable
  model.x = Var(range(num_stocks), domain=NonNegativeReals)
 
  #defining the objective function
  model.Objective = Objective(expr = sum(future_price[i]*model.x[i]-(current_price[i]*model.x[i]) for i in range(num_stocks)), sense=maximize)

  #Adding the constraints
  model.Budget = Constraint(expr = sum(current_price[i]*model.x[i] for i in range(num_stocks)) <= budget)
  total_invested = sum(current_price[i]*model.x[i] for i in range(num_stocks))
  final_return = sum(((future_price[i]-current_price[i])*model.x[i]) for i in range(num_stocks))
  model.AvgRet = Constraint(expr = final_return/total_invested >= return_min)
  model.valueConst=ConstraintList()
  for i in range(num_stocks):
    model.valueConst.add(current_price[i]*model.x[i] <= indv_budget[i])

  #solving the model
  opt = SolverFactory('ipopt')
  opt.solve(model, tee=False)

  optimal_portfolio = [round(current_price[i]*model.x[i](),2) for i in range(num_stocks)]
  optimal_profit = round(model.Objective(),2)
  optimal_invested = sum(current_price[i]*model.x[i]() for i in range(num_stocks))
  optimal_units = [round(model.x[i](),2) for i in range(num_stocks)]
  optimal_returns = optimal_profit/optimal_invested
  optimal_prof_comp = [round((future_price[i]-current_price[i])*model.x[i](),2) for i in range(num_stocks)]

  return[optimal_prof_comp,optimal_profit,optimal_returns,optimal_units]

In [ ]:
company = df['Company'].tolist()
current_price = df['Current_Price'].tolist()
future_price = df['Future_Price'].tolist()

Following list provides answers to our optimization model.

In [ ]:
solution = solveproblem(company,all_sector,current_price,future_price)

In [ ]:
df['optimal_units'] = solution[3]
df['optimal_profits'] = solution[0]
df

,Company,Sector,Current_Price,Future_Price,Returns,optimal_units,optimal_profits
0,TCS,IT services,41.090854,41.835366,0.018119,0.00,0.00
1,Infosys,IT services,19.777439,20.121951,0.017419,0.00,0.00
2,Wipro,IT services,4.962195,5.040854,0.015852,0.00,0.00
3,Bajaj Finance,Finance,80.551220,82.314024,0.021884,62.07,109.42
4,L&T Finance,Finance,1.126829,1.140244,0.011905,0.00,0.00
5,HDFC AMC,Finance,27.285366,27.421341,0.004983,0.00,0.00
6,NTPC,Power Generation,2.099390,2.128049,0.013651,0.00,0.00
7,Power Grid,Power Generation,2.705488,2.760366,0.020284,1848.10,101.42
8,Tata Power,Power Generation,2.753659,2.817073,0.023029,1815.77,115.15
9,HDFC Bank,Private Banks,19.589634,19.967073,0.019267,0.00,0.00


In [ ]:
df.to_csv('optimal_portfolio.csv', index=False)
files.download("optimal_portfolio.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>